# デバイス一覧を表示

https://codeday.me/jp/qa/20190429/731178.html

In [1]:
import pyaudio
import pprint

pa = pyaudio.PyAudio()
chosen_device_index = -1
for x in range(0, pa.get_device_count()):
    info = pa.get_device_info_by_index(x)
    pprint.pprint(pa.get_device_info_by_index(x))
    if info["name"] == "pulse":
        chosen_device_index = info["index"]
        print("Chosen index: ", chosen_device_index)

{'defaultHighInputLatency': -1.0,
 'defaultHighOutputLatency': 0.034829931972789115,
 'defaultLowInputLatency': -1.0,
 'defaultLowOutputLatency': 0.005804988662131519,
 'defaultSampleRate': 44100.0,
 'hostApi': 0,
 'index': 0,
 'maxInputChannels': 0,
 'maxOutputChannels': 8,
 'name': 'HDA Intel HDMI: 0 (hw:0,3)',
 'structVersion': 2}
{'defaultHighInputLatency': -1.0,
 'defaultHighOutputLatency': 0.034829931972789115,
 'defaultLowInputLatency': -1.0,
 'defaultLowOutputLatency': 0.005804988662131519,
 'defaultSampleRate': 44100.0,
 'hostApi': 0,
 'index': 1,
 'maxInputChannels': 0,
 'maxOutputChannels': 8,
 'name': 'HDA Intel HDMI: 1 (hw:0,7)',
 'structVersion': 2}
{'defaultHighInputLatency': -1.0,
 'defaultHighOutputLatency': 0.034829931972789115,
 'defaultLowInputLatency': -1.0,
 'defaultLowOutputLatency': 0.005804988662131519,
 'defaultSampleRate': 44100.0,
 'hostApi': 0,
 'index': 2,
 'maxInputChannels': 0,
 'maxOutputChannels': 8,
 'name': 'HDA Intel HDMI: 2 (hw:0,8)',
 'structVersi

"pulse"と名前のついたデバイスを選択する 

# 正弦波の再生

http://aidiary.hatenablog.com/entry/20110607/1307449007

In [18]:
import wave
import struct
import numpy as np
from pylab import *


def createSineWave(A, f0, fs, length):
    """振幅A、基本周波数f0、サンプリング周波数 fs、
    長さlength秒の正弦波を作成して返す"""
    data = []
    # [-1.0, 1.0]の小数値が入った波を作成
    for n in arange(length * fs):  # nはサンプルインデックス
        s = A * np.sin(2 * np.pi * f0 * n / fs)
        # 振幅が大きい時はクリッピング
        if s > 1.0:
            s = 1.0
        if s < -1.0:
            s = -1.0
        data.append(s)
    # [-32768, 32767]の整数値に変換
    data = [int(x * 32767.0) for x in data]
    #    plot(data[0:100]); show()
    # バイナリに変換
    data = struct.pack("h" * len(data), *data)  # listに*をつけると引数展開される
    return data


def play(data, fs, bit):
    import pyaudio
    # ストリームを開く
    p = pyaudio.PyAudio()
    stream = p.open(
        format=pyaudio.paInt16,
        channels=1,
        output_device_index=chosen_device_index,
        rate=int(fs),
        output=True,
    )
    # チャンク単位でストリームに出力し音声を再生
    chunk = 1024
    sp = 0  # 再生位置ポインタ
    buffer = data[sp : sp + chunk]
    while buffer != b"":
        stream.write(buffer)
        sp = sp + chunk
        buffer = data[sp : sp + chunk]
    stream.close()
    p.terminate()


if __name__ == "__main__":
    freqList = [262, 294, 330, 349, 392, 440, 494, 523]  # ドレミファソラシド
    for f in freqList:
        data = createSineWave(1.0, f, 8000.0, 1.0)
        play(data, 8000, 16)